In [1]:
from toolz.curried import *
import pandas as pd

In [24]:
sp500 = pd.read_csv("sp500.csv").astype({"date":"datetime64[ns]"})
selic = pd.read_csv("selic.csv").astype({"date":"datetime64[ns]"})
dolar = pd.read_csv("brl_usd.csv").astype({"date":"datetime64[ns]"})
ipca = pd.read_csv("ipca.csv").astype({"date":"datetime64[ns]"})

## Merge on SP500 Data

In [106]:
daily_data = (sp500
              .merge(dolar)
              .assign(month=lambda d: d["date"].dt.to_period("M"))
              .assign(sp500=lambda d: d["sp500"] * d["dolar"]) # convert sp500 to BRL
              .drop(columns=["dolar"]))

daily_data.head()

,date,sp500,month
0,1994-01-27,76.76539,1994-01
1,1994-01-28,78.36319,1994-01
2,1994-01-31,80.33088,1994-01
3,1994-02-01,81.53200,1994-02
4,1994-02-02,83.48240,1994-02


In [104]:
month_start = (
    daily_data
    .assign(
        start_date = lambda d: d.groupby("month")["date"].transform("min"),
    )
    .query("date==start_date")
    .set_index("month")
    [["sp500"]]
)

month_end = (
    daily_data
    .assign(
        end_date = lambda d: d.groupby("month")["date"].transform("max"),
    )
    .query("date==end_date")
    .set_index("month")
    [["sp500"]]
)


monthly_growth = (((month_end - month_start)/month_start)
                  .reset_index()
                  .assign(date=lambda d:d["month"].dt.to_timestamp())
                  .drop(columns="month"))

monthly_growth.head()

,sp500,date
0,0.046447,1994-01-01
1,0.327418,1994-02-01
2,0.354286,1994-03-01
3,0.437367,1994-04-01
4,0.427734,1994-05-01


In [105]:
data = (selic
        .merge(ipca, how="left")
        .merge(monthly_growth)
        .query("date>='1995-01-01'")
        .query("date<'2022-12-01'"))

data.head()

,date,selic,selic_aa,ipca,sp500
6,1995-01-01,0.033697,0.4884,0.0170,0.020364
7,1995-02-01,0.032497,0.4678,0.0102,0.048467
8,1995-03-01,0.042598,0.6497,0.0155,0.091106
9,1995-04-01,0.042598,0.6497,0.0243,0.040242
10,1995-05-01,0.042498,0.6478,0.0267,0.024690


In [94]:
data

,date,selic,selic_aa,ipca,sp500,dolar
6,1995-01-01,0.033697,0.4884,0.0170,0.024613,-0.004147
7,1995-02-01,0.032497,0.4678,0.0102,0.036139,0.011898
8,1995-03-01,0.042598,0.6497,0.0155,0.031096,0.058201
9,1995-04-01,0.042598,0.6497,0.0243,0.025503,0.014373
10,1995-05-01,0.042498,0.6478,0.0267,0.037138,-0.012002
...,...,...,...,...,...,...
336,2022-07-01,0.010296,0.1308,-0.0068,0.079721,-0.029563
337,2022-08-01,0.011700,0.1498,-0.0036,-0.039729,-0.000135
338,2022-09-01,0.010698,0.1362,-0.0029,-0.096104,0.033158
339,2022-10-01,0.010200,0.1295,0.0059,0.052618,0.002303


## Simulation

In [114]:
from toolz.curried import *
from typing import Dict


def withdraw(r: float,
             inflation: float,
             weights: Dict[str, float],
             portfolio: Dict[str, float],
             tax: float = 0.15):

    total = sum(portfolio.values())
    withdraw_amount = total * r * (1 + inflation) / (1 - tax)

    w_diff = {a: (w / total) - weights.get(a) for a, w in portfolio.items()}
    w_diff = sorted(w_diff.items(), key=lambda x: x[1], reverse=True)

    total_to_withdraw = withdraw_amount
    updated_portfolio = dict()

    for asset, _ in w_diff:
        to_withdraw = min(total_to_withdraw, portfolio.get(asset))
        updated_portfolio[asset] = portfolio.get(asset) - to_withdraw
        total_to_withdraw -= to_withdraw

    return {"amount": withdraw_amount * (1 - tax),
            "portfolio": updated_portfolio}


def grow(portfolio: Dict[str, float],
         asset_growth: Dict[str, float]):
    return {asset: value*(1+asset_growth.get(asset))
            for asset, value in portfolio.items()}



def rebalance(portfolio, weights, tax=0.15):
    
    if any([v>0 for v in weights.values()]):
        return portfolio
    
    total = sum(portfolio.values())
    w_diff = {a:(w/total)-weights.get(a) for a, w in portfolio.items()}
    withdraw_from = max(w_diff, key=w_diff.get)
    withdraw_to = min(w_diff, key=w_diff.get)
    
    A1 = portfolio.get(withdraw_from)
    A2 = total - A1
    w1 = weights.get(withdraw_from)
    
    A1_ = (A1+A2-A1*tx)/((1/w1) - tax)
    A2_ = A1_/w1 - A1_
    
    return {withdraw_from: A1_, withdraw_to: A2_}

### Test Functions

In [109]:
balance = 1000000
weights = {"selic": 0.5, "sp500": 0.5}
portfolio = {asset: w*balance for asset, w in weights.items()}
# portfolio = {"selic": 0.3*balance, "sp500": 0.7*balance}
anual_wr = 0.04
monthly_wr = anual_wr/12

portfolio

{'selic': 500000.0, 'sp500': 500000.0}

In [116]:
m1 = data.iloc[0].to_dict()
m1

{'date': Timestamp('1995-01-01 00:00:00'),
 'selic': 0.03369711721997,
 'selic_aa': 0.4884,
 'ipca': 0.017,
 'sp500': 0.02036438488243523}

In [111]:
new_portfolio = withdraw(monthly_wr, m1.get("ipca"), weights, portfolio)
new_portfolio

{'amount': 3390.0,
 'portfolio': {'selic': 496011.76470588235, 'sp500': 500000.0}}

In [118]:
new_portfolio_g = grow(new_portfolio.get("portfolio"),
                       asset_growth=m1)

new_portfolio_g

{'selic': 512725.93128366064, 'sp500': 510182.19244121766}

In [370]:
rebalance(new_portfolio_g, weights)

{'selic': 0.0, 'sp500': 1016294.9316297865}

In [371]:
balance = 3000000
weights = {"selic": 0.4, "sp500": 0.6}
portfolio = {asset: w*balance for asset, w in weights.items()}
anual_wr = 0.04
monthly_wr = anual_wr/12

portfolio

{'selic': 1200000.0, 'sp500': 1800000.0}

In [372]:
data.head(10)

,date,selic,selic_aa,ipca,sp500,dolar
0,1995-01-01,0.033697,0.4884,0.0170,0.024613,-0.004147
1,1995-02-01,0.032497,0.4678,0.0102,0.036139,0.011898
2,1995-03-01,0.042598,0.6497,0.0155,0.031096,0.058201
3,1995-04-01,0.042598,0.6497,0.0243,0.025503,0.014373
4,1995-05-01,0.042498,0.6478,0.0267,0.037138,-0.012002
5,1995-06-01,0.040398,0.6084,0.0226,0.021181,0.014120
6,1995-07-01,0.040198,0.6047,0.0236,0.027417,0.017187
7,1995-08-01,0.038399,0.5717,0.0099,0.004110,0.015079
8,1995-09-01,0.033198,0.4798,0.0099,0.036538,0.003369
9,1995-10-01,0.030901,0.4408,0.0141,-0.000344,0.003549


In [332]:
balance = 1000000
weights = {"selic": 0.4, "sp500": 0.6}
portfolio = {asset: w*balance for asset, w in weights.items()}
# portfolio = {"selic": 0.3*balance, "sp500": 0.7*balance}
anual_wr = 0.04
monthly_wr = anual_wr/12

portfolio

{'selic': 400000.0, 'sp500': 600000.0}

In [435]:
sim_result = simulate_portfolio_evol(
    balance=498261,
    weights = {"selic": 0.3, "sp500": .7},
    anual_wr = 0.04,
    growth_df=data
)

sim_result
# pd.to_datetime(sim_result["date"])

,selic,sp500,ammount,total,date,ipca,dolar_g,sp500_g,selic_g
0,149478.30,348782.70,0.00,498261.00,1900-01-01,0.0000,0.000000,0.000000,0.000000
1,152461.14,355885.45,1689.10,508346.59,1995-01-01,0.0170,-0.004147,0.024613,0.033697
2,157415.71,371022.73,1711.77,528438.44,1995-02-01,0.0102,0.011898,0.036139,0.032497
3,164121.38,402529.12,1788.76,566650.50,1995-03-01,0.0155,0.058201,0.031096,0.042598
4,171112.70,416360.03,1934.73,587472.73,1995-04-01,0.0243,0.014373,0.025503,0.042598
...,...,...,...,...,...,...,...,...,...
331,1257215.29,4094400.19,17123.29,5351615.48,2022-07-01,-0.0068,-0.029563,0.079721,0.010296
332,1271925.02,3911124.22,17774.50,5183049.24,2022-08-01,-0.0036,-0.000135,-0.039729,0.011700
333,1285531.56,3633544.40,17226.73,4919075.96,2022-09-01,-0.0029,0.033158,-0.096104,0.010698
334,1298643.45,3813068.59,16493.66,5111712.04,2022-10-01,0.0059,0.002303,0.052618,0.010200


In [415]:
pd.Timestamp("1995-01-01").month

1

In [436]:
498261.00*(1+5.02)

2999531.2199999997

In [419]:
def simulate_portfolio_evol(balance,
                            weights,
                            anual_wr,
                            growth_df,
                            rebalance_on_month=12,
                            tax=0.15):
    
    portfolio0 = {asset: w*balance for asset, w in weights.items()}
    monthly_wr = anual_wr/12
    
    portifolios = [portfolio0]
    
    result = [merge(
        portfolio0,
        {"ammount": 0,
         "total": balance,
         "date": pd.to_datetime("1900-01-01"),
         "ipca": 0,
         "dolar_g": 0,
         "sp500_g": 0,
         "selic_g": 0}
    )]
        
        
    for _, row in growth_df.iterrows():
        month = row.to_dict()
        p_prev = portifolios[-1]
        
        withdraw_result = withdraw(monthly_wr, month.get("ipca"), weights, p_prev, tax=tax)
        
        p_grow = grow(withdraw_result.get("portfolio"),
                      selic=month.get("selic"),
                      sp500=month.get("sp500"),
                      dolar=month.get("dolar"))
        
        if month.get("date").month == rebalance_on_month:
            p_grow = rebalance(p_grow, weights, tax=tax)
        
        portifolios.append(p_grow)
        
        result_dict = merge(p_grow,
                            {"ammount": withdraw_result.get("ammount"),
                             "total": sum(portifolios[-1].values()),
                             "date": month.get("date"),
                             "ipca": month.get("ipca"),
                             "dolar_g": month.get("dolar"),
                             "sp500_g": month.get("sp500"),
                             "selic_g": month.get("selic")})
        
        result.append(result_dict)
    
    
        
    return pd.DataFrame(result).round({"selic":2, "sp500":2, "ammount":2, "total":2})



In [209]:
portifolios = [portfolio]
result = [merge(
    portfolio,
    {"ammount": 0,
     "total": balance,
     "date": 0,
     "ipca": 0,
     "dolar_g": 0,
     "sp500_g": 0,
     "selic_g": 0,}
)]

for _, row in growth_data.iterrows():
    month = row.to_dict()
    p_prev = portifolios[-1]
    
    withdraw_result = withdraw(monthly_wr, month.get("ipca"), weights, p_prev)
    p_grow = grow(withdraw_result.get("portfolio"),
                  selic=month.get("selic"), sp500=month.get("sp500"), dolar=month.get("dolar"))
    
    portifolios.append(p_grow)
    result_dict = merge(p_grow,
                        {"ammount": withdraw_result.get("ammount"),
                         "total": sum(p_grow.values()),
                         "date": month.get("date"),
                         "ipca": month.get("ipca"),
                         "dolar_g": month.get("dolar"),
                         "sp500_g": month.get("sp500"),
                         "selic_g": month.get("selic")})
    
    result.append(result_dict)
    


In [362]:

v0 = 50
d0 = 5
vd0 = v0/d0

print(vd0)

g = 0.01
dg = 0.01

vd0*(1+g) * (d0*(1+dg))

10.0


51.004999999999995

In [364]:
v0*(1+dg)*(1+g)

51.005

In [428]:
6020935.40/1000000 - 1

5.0209354

In [431]:
3000000/(5.0209354+1)

498261.44954154466

$$
M_f = \sum_{t}^T A(1+r)^{(5*12 - t)}
$$

$$
M_f*r = A
$$


$$
M_f = A/r
$$



In [241]:
def check_r(r):
    A = 100
    Mf = sum([A*(1+r)**(5*12-t) for t in range(1, 5*12+1)])
    needed_mf = A/r
    print(Mf, needed_mf)
    return Mf >= needed_mf

In [255]:
check_r(anual_to_monthly_rate(0.04))

6617.902362859201 30546.105265835064


False

In [257]:
check = False
r = 0.04
while not(check):
    check = check_r(anual_to_monthly_rate(r))
    print(r, check)
    r += 0.01

6617.902362859201 30546.105265835064
0.04 False
6781.3737915590655 24545.15505919425
0.05 False
6948.578613813823 20544.213903395237
0.060000000000000005 False
7119.585775008405 17686.13877930201
0.07 False
7294.465036312172 15542.358100644122
0.08 False
7473.286978732578 13874.776475769677
0.09 False
7656.123007153155 12540.536612263722
0.09999999999999999 False
7843.04535435653 11448.729273519348
0.10999999999999999 False
8034.127085032721 10538.748256960902
0.11999999999999998 False
8229.442099772983 9768.635382262868
0.12999999999999998 False
8429.065139050079 9108.420484568072
0.13999999999999999 False
8633.071787184886 8536.125410375644
0.15 True
